In [1]:
import json
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split 
import requests

In [3]:
# load dataset
df = pd.read_csv(r'/home/patrick/Github/Mosaic_DDOS_Datasets/train_mosaic.csv')
x_cols = [c for c in df.columns if c != 'Label']
# set input matrix and target column
X = df[x_cols]
y = df['Label']
# show first rows of data
df.head()

,Destination_Port,Flow_Duration,Total_Fwd_Packets,Total_Backward_Packets,Total_Length_of_Fwd_Packets,Total_Length_of_Bwd_Packets,Fwd_Packet_Length_Max,Fwd_Packet_Length_Min,Fwd_Packet_Length_Mean,Fwd_Packet_Length_Std,...,min_seg_size_forward,Active_Mean,Active_Std,Active_Max,Active_Min,Idle_Mean,Idle_Std,Idle_Max,Idle_Min,Label
0,80,101168794,20,1,969,0,353,0,48.450000,119.083551,...,0.0,739228.5,743103.4661,1264682.0,213775.0,49700000.0,41400000.0,79000000.0,20500000.0,DoS Hulk
1,60711,58,1,1,0,0,0,0,0.000000,0.000000,...,32.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,53,31146,4,2,148,244,37,37,37.000000,0.000000,...,20.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,80,254704,3,4,429,389,423,0,143.000000,242.505670,...,20.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,443,11932077,12,16,5030,15703,1525,0,419.166667,644.896586,...,20.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN


In [4]:
# Split the dataset into a training set and a validation set.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=1234)

In [13]:
for i in range(100):
    input_data = dict(X_test.iloc[i])
    target = y_test.iloc[i]
    r = requests.post("http://127.0.0.1:8000/api/v1/ddos_classifier/predict?status=ab_testing", input_data)
    response = r.json()
    print("Response")
    print(response)
    # provide feedback
    requests.put("http://127.0.0.1:8000/api/v1/mlrequests/{}".format(response["request_id"]), {"feedback": target})

Response
{'probability': 1.0, 'label': 'BENIGN', 'status': 'OK', 'request_id': 102}
Response
{'probability': 0.0, 'label': 'DDOS', 'status': 'OK', 'request_id': 103}
Response
{'probability': 0.0, 'label': 'DDOS', 'status': 'OK', 'request_id': 104}
Response
{'probability': 1.0, 'label': 'BENIGN', 'status': 'OK', 'request_id': 105}
Response
{'probability': 0.0, 'label': 'DDOS', 'status': 'OK', 'request_id': 106}
Response
{'probability': 1.0, 'label': 'BENIGN', 'status': 'OK', 'request_id': 107}
Response
{'probability': 1.0, 'label': 'BENIGN', 'status': 'OK', 'request_id': 108}
Response
{'probability': 0.0, 'label': 'DDOS', 'status': 'OK', 'request_id': 109}
Response
{'probability': 0.0, 'label': 'DDOS', 'status': 'OK', 'request_id': 110}
Response
{'probability': 0.0, 'label': 'DDOS', 'status': 'OK', 'request_id': 111}
Response
{'probability': 0.0, 'label': 'DDOS', 'status': 'OK', 'request_id': 112}
Response
{'probability': 0.0, 'label': 'DDOS', 'status': 'OK', 'request_id': 113}
Response